# Base de datos MongoDB

## Lectura de datos y desnormalizacion

In [136]:
import pandas as pd

df_dificultades = pd.read_csv("./data/dificultades.csv",encoding='utf-8');
df_encadenamientos = pd.read_csv("./data/tipos_encadenamiento.csv", encoding='utf-8');
df_escaladores = pd.read_csv("./data/escaladores_lite_2017.csv", encoding='utf-8');
df_ascensos = pd.read_csv("./data/ascensos_lite_2017.csv", encoding='utf-8');

df_ascensos = pd.merge(df_ascensos, df_encadenamientos, on = ['id_tipo_encadenamiento'], how = 'inner')
df_ascensos = pd.merge(df_ascensos, df_dificultades, on = ['id_dificultad'], how = 'inner');
df_ascensos = pd.merge(df_ascensos, df_escaladores, on = ['id_escalador'], how = 'inner');

# Renombrar campos por claridad (el merge auto-asigna nombres a campos repetidos)
df_ascensos = df_ascensos.rename(
    columns={
        'nombre': 'nombre_escalador',
        'ciudad':'ciudad_escalador',
        'pais_y': 'pais_escalador',
        'comienzo': 'anio_comienzo',
        'pais_x': 'pais_risco'});


# Filtro las columnas de ascensos dejando solo aquellas que interesan
columnas = ['id_escalador', 'nombre_escalador','pais_escalador', 'anio_comienzo',
            'nombre_via', 'id_dificultad', 'grado_frances', 'tipo_encadenamiento',
            'risco', 'pais_risco']

df_ascensos = df_ascensos[columnas]
df_ascensos.head(3)

,id_escalador,nombre_escalador,pais_escalador,anio_comienzo,nombre_via,id_dificultad,grado_frances,tipo_encadenamiento,risco,pais_risco
0,20384,Clemens Kurth,DEU,2003,LE DE,33,5c,Flash,FONTAINEBLEAU,FRA
1,20384,Clemens Kurth,DEU,2003,SCHMALSEITE DV,33,5c,Flash,ELBSANDSTEIN,DEU
2,20384,Clemens Kurth,DEU,2003,TALSEITE,33,5c,Onsight,ELBSANDSTEIN,DEU


## Conexión a Mongo

In [137]:
import pymongo
from pymongo import MongoClient

from pymongo import MongoClient
client = MongoClient('localhost', 27017)

client.drop_database("practica_8anu")

db = client.practica_8anu

## Creacion de collecciones MongoDB

Las preguntas planteadas van en dos direcciones:
- Dados los escaladores, preguntas sobre sus ascensos.
- Preguntas sobre los ascensos en si mismos.

MongoDB no ofrece la posibilidad de hacer Joins de collecciones. Para resolver esas dos direcciones, me viene bien plantear dos colecciones con la siguiente información:

** TO-DO **

In [138]:
df_escaladores.head()

,id_escalador,nombre,sexo,fecha_nacimiento,ciudad,pais,comienzo
0,28,Knut Rokne,Hombre,1972-03-27,Calgary,CAN,1988
1,38,Alan Cassidy,Hombre,1982-12-10,Glasgow,GBR,1993
2,408,André Neres,Hombre,1985-09-08,Lisboa,PRT,2001
3,482,Adam Strong,Hombre,1972-04-10,Estes Park,USA,0
4,574,Mike Forward,Hombre,1981-11-03,Sydney,AUS,1991


### Coleccion de escaladores

In [139]:
import json

def inserta_escalador(escalador_json):
    escalador_json['_id']=escalador_json['id_escalador']
    del escalador_json['id_escalador']
    db.escaladores.insert_one(escalador_json)

    
json_string = df_escaladores.to_json(orient = 'records')
lista_json = json.loads(json_string)

# añade los escaladores a la colección    
for escalador_json in lista_json :
    inserta_escalador(escalador_json)

In [5]:
df_ascensos.head(2)

,id_escalador,nombre_escalador,pais_escalador,nombre_via,id_dificultad,grado_frances,tipo_encadenamiento,risco,pais_risco
0,20384,Clemens Kurth,DEU,LE DE,33,5c,Flash,FONTAINEBLEAU,FRA
1,20384,Clemens Kurth,DEU,SCHMALSEITE DV,33,5c,Flash,ELBSANDSTEIN,DEU


### Coleccion de ascensos

In [140]:

def inserta_ascenso(ascenso_json):
    #registramos el ascenso en la collección de la bb.dd.
    insert_result = db.ascensos.insert_one(ascenso_json)
    
    #suma uno al contador de ascensos del escalador y actualiza su coleccion de ascensos 
    db.escaladores.find_and_modify(query = {"_id" : ascenso_json['id_escalador']},
                                   update ={ "$inc": { "numero_ascensos": 1,
                                                       "dificultad_acumulada": ascenso_json["id_dificultad"]},
                                             "$push": {'ascensos' : insert_result.inserted_id}
                                           })
    

json_string = df_ascensos.to_json(orient = 'records')
lista_json = json.loads(json_string)

# añade los escaladores a la colección    
for ascenso_json in lista_json :
    inserta_ascenso(ascenso_json)

### Coleccion de dificultades

In [141]:

def inserta_dificultad(dificultad_json):
    dificultad_json['_id']=dificultad_json['id_dificultad']
    del dificultad_json['id_dificultad']
    db.dificultades.insert_one(dificultad_json)

    
json_string = df_dificultades.to_json(orient = 'records')
lista_json = json.loads(json_string)

# añade los escaladores a la colección    
for dificultad_json in lista_json :
    inserta_dificultad(dificultad_json)

## Respuestas a las preguntas

### 1.a) Los 10 escaladores (hombres) más activos (orden auxiliar por Id)

In [8]:
result = db.escaladores.find(
    {
        "sexo" : "Hombre"
    },
    {
        "_id": 1,
        "nombre": 1,
        "sexo": 1,
        "pais": 1,
        "numero_ascensos": 1
    }).sort([("numero_ascensos", -1), ("_id", -1)]).limit(10)

df = pd.DataFrame(list(result))
df

,_id,nombre,numero_ascensos,pais,sexo
0,50884,Christopher Leonetti,47,USA,Hombre
1,20384,Clemens Kurth,26,DEU,Hombre
2,20095,Matthias Schuster,26,DEU,Hombre
3,66250,Kuba Kaminski,24,POL,Hombre
4,9171,Laurenz Trawnicek,23,AUT,Hombre
5,46622,Raúl Crespo,19,ESP,Hombre
6,42086,Marcin Opozda,19,POL,Hombre
7,66466,Christian Boehme,18,DEU,Hombre
8,47732,Thomas de Fleurian,18,FRA,Hombre
9,35847,Zack Bum,18,DEU,Hombre


In [9]:
escalador_mas_activo = df['_id'].iloc[0]
print('Escalador mas activo',escalador_mas_activo)

('Escalador mas activo', 50884)


### 1.b) Los 10 escaladoras (mujeres) más activas (orden auxiliar por Id)

In [10]:
result = db.escaladores.find(
    { 
        "sexo" : "Mujer"
    },
    {
        "_id": 1,
        "nombre": 1,
        "sexo": 1,
        "pais": 1,
        "numero_ascensos": 1
    }).sort([("numero_ascensos", -1), ("_id", -1)]).limit(10)

df = pd.DataFrame(list(result))
df

,_id,nombre,numero_ascensos,pais,sexo
0,65502,Ksenia Targosz,13,POL,Mujer
1,65069,Mania Mania A.,12,POL,Mujer
2,65707,Ewelina Cienkus,11,POL,Mujer
3,54695,Elfi Hasler,11,AUT,Mujer
4,62354,La Shoune,10,FRA,Mujer
5,58835,Dominika Sołtys,10,POL,Mujer
6,57447,Sue Murphy,10,CAN,Mujer
7,53983,Daniela Bärtschi,10,CHE,Mujer
8,49569,Karina Kosiorek,10,POL,Mujer
9,32043,Reidun M. Romundstad,10,NOR,Mujer


In [11]:
escaladora_mas_activa = df['_id'].iloc[0]
print('Escaladora mas activa',escaladora_mas_activa)

('Escaladora mas activa', 65502)


### 2. Lista de los 10 ascensos "On sight" de la escaladora más activa en orden decreciente de dificultad (y por nombre de via ascendete)

In [12]:
result = db.ascensos.find(
    { 
        "id_escalador" : escaladora_mas_activa,
        "tipo_encadenamiento" : "Onsight"
    },
    {
        "_id" : 0,
        "nombre_via": 1,
        "grado_frances": 1,
        "risco": 1,
        "pais": 1,
    }).sort([("id_dificultad", -1), ("nombre_via", 1)]).limit(10)

df = pd.DataFrame(list(result))
df

,grado_frances,nombre_via,risco
0,5c,LEWIZNA,DOLINA KOBYLANSKA
1,5c,MOJA KOKAINA,DOLINA KOBYLANSKA
2,5c,RUCHY MASOWE,DOLINA BEDKOWSKA
3,5b,FISCHER-LAPINSKI,DOLINA BEDKOWSKA
4,5b,GRZBIET ZUBRA,SOKOLIKI
5,5b,RYSKOWIEC,DOLINA BEDKOWSKA
6,4c,ZAGłADA TRADA,DOLINA KOBYLANSKA
7,4b,KIKUT,DOLINA SZKLARKI


### 3. Dificultad media de los ascensos del escalador más activo

MongoDB no ofrece la posibilidad de hacer joins entre la tablas de ascensos y la de escaladores, por lo que hay tres posibilidades (al menos que se me ocurran) para responder a esta pregunta:

- una posibilidad incluir en el usuario un campo de acumulación de dificultad. Esto implica actulizar ese campo de acumulación cuando hay unaentrada de un ascenso...

- la otra opcion es obtener todos los ascensos del escalador (que tenemos como un array embebido) y delegar en el codigo cliente (pythoin) el iterar en ellas para obtener la media.

- utilizar la capacidad de agregacion que ofrece MongoDB. Agregaremos la información de los ascensos del escalador obteniendo directamete la media por consulta.

Aunque tenemos preparada la colección de escaladores para llevar a cabo cualquiera de las dos opciones, elijo la tecera, ya que es funcionalidad que me ofrece la base de datos y me libera de hacer calculos en el cliente.

In [126]:
from bson.son import SON

#obtenemos los ascensos en España de los escaladores españoles
result = db.ascensos.aggregate([
    { "$match" : { "id_escalador": escalador_mas_activo } },
    { "$group" : {  "_id": "$id_escalador",
                    "dificultad_media_float":{ "$avg": "$id_dificultad"},
                    "nombre_escalador" :  {"$addToSet" : "$nombre_escalador"},
                 }},
    { "$unwind" : "$nombre_escalador" }, 
])
df = pd.DataFrame(list(result))



**MongoDB no ofrece un operador de redondeo *$round* para las agregaciones**.  Por ello, aunque se podría hacer una combinacion de operadores aritmeticos, es mas sencillo añadir una columna al data frame resultante para obtener un redondeo.

In [127]:
df['dificultad_media'] = df['dificultad_media_float'].round().astype(int)

**MongoDB no proporciona un funcionalidad de join** de tablas, por lo que, **para que incluir la información con el grado frances debo hacerlo en el cliente** (código python). Para ello voy a crear una funcion que haga ese mapeo, ya que lo haré varias veces a lo largo del notebook. Esta función:
- tomara de entrada el data frame, las columnas en las está la dificultad a mapear y las columnas que se crearan con el grado mapeado
-leerá la coleccion de dificultades completa (que afortunadamente es pequeña...) asignango el indice al id_dificultad
- creara tantas columnas nuevas como se especifiquen mapeando las de dificultad que se le han pasado por parametro

In [128]:
# funcion para hacer el mapeo de columnas de dificultad a columnas de grado francés
def mapeaDificultadAGradoFrances(data_frame, columnas_origen, columnas_destino) :
    # obtengo data frame de dificultades poniendo el id de dificultad como indice
    result = db.dificultades.find( { }, { "_id" : 1, "grado_frances": 1})
    df_dificultades = pd.DataFrame(list(result))
    df_dificultades=df_dificultades.set_index('_id');
    
    for columna_origen, columna_destino in zip(columnas_origen, columnas_destino) :
        data_frame[columna_destino] = data_frame[columna_origen].map(df_dificultades['grado_frances'])    

In [129]:
mapeaDificultadAGradoFrances(df, ['dificultad_media'], ['grado_medio'])

#muestra los resultados
columnas = ['nombre_escalador', 'grado_medio']
df[columnas]

,nombre_escalador,grado_medio
0,Christopher Leonetti,5c+


### 4.a) Los 10 ascensos mas dificiles

Al igual que en la anterior, obtener esta información con los datos del usuario requeriría un join. Lo voy a plantear de manera que, desnormalizando la información de ascensos, ya incluyamos esto en los documentos 'ascenso'. Con ello, logro tener un solo paso de cliente (python) consistente en una consulta, si tener que hacer un join manual externo.

In [14]:
result = db.ascensos.find(
    { },
    {
        "_id" : 0,
        "id_escalador" : 1,
        "nombre_escalador" : 1,
        "pais_escalador" : 1,
        "nombre_via" : 1,
        "grado_frances" : 1,
        "tipo_encadenamiento" : 1,
        "risco": 1,
        "pais_risco": 1,
    }).sort([("id_dificultad", -1), ("id_escalador", 1)]).limit(10)

df = pd.DataFrame(list(result));
columnas = ['id_escalador','nombre_escalador','pais_escalador','nombre_via','grado_frances','tipo_encadenamiento','risco','pais_risco'];
df[columnas]

,id_escalador,nombre_escalador,pais_escalador,nombre_via,grado_frances,tipo_encadenamiento,risco,pais_risco
0,1476,Adam Ondra,CZE,SILENCE,9c,Redpoint,FLATANGER,NOR
1,1476,Adam Ondra,CZE,MOVE HARD,9b,Redpoint,FLATANGER,NOR
2,22437,Stefano Ghisolfi,ITA,FIRST ROUND FIRST MINUTE,9b,Redpoint,MARGALEF,ESP
3,1476,Adam Ondra,CZE,ULTIMATUM,9a+,Redpoint,ARCO,ITA
4,1476,Adam Ondra,CZE,NATURALMENTE,9a+,Redpoint,CAMAIORE,ITA
5,8707,Daniel Fuertes,ESP,NO PAIN NO GAIN,9a+,Redpoint,RODELLAR,ESP
6,14130,David Firnenburg,DEU,LA RAMBLA,9a+,Redpoint,SIURANA,ESP
7,18008,Piotr Schab,POL,THOR'S HAMMER,9a+,Redpoint,FLATANGER,NOR
8,22437,Stefano Ghisolfi,ITA,FIRST LEY,9a+,Redpoint,MARGALEF,ESP
9,22437,Stefano Ghisolfi,ITA,LA RAMBLA,9a+,Redpoint,SIURANA,ESP


### 4.b) Los 10 ascensos mas dificiles a vista (On sight)

In [15]:
result = db.ascensos.find(
    { 
        "tipo_encadenamiento": "Onsight" 
    },
    {
        "_id" : 0,
        "id_escalador" : 1,
        "nombre_escalador" : 1,
        "pais_escalador" : 1,
        "nombre_via" : 1,
        "grado_frances" : 1,
        "tipo_encadenamiento" : 1,
        "risco": 1,
        "pais_risco": 1,
    }).sort([("id_dificultad", -1), ("id_escalador", 1)]).limit(10)

df = pd.DataFrame(list(result));

#defino las columnas para mostrarlas en un orden deseado
columnas = ['id_escalador','nombre_escalador','pais_escalador','nombre_via','grado_frances','tipo_encadenamiento','risco','pais_risco'];
df[columnas]

,id_escalador,nombre_escalador,pais_escalador,nombre_via,grado_frances,tipo_encadenamiento,risco,pais_risco
0,22437,Stefano Ghisolfi,ITA,FISH EYE,8c,Onsight,OLIANA,ESP
1,18008,Piotr Schab,POL,PEQUENA ESTRELLA,8b+,Onsight,RODELLAR,ESP
2,6726,Manu Lopez,FRA,WHAT,8b,Onsight,LEONIDIO,GRC
3,38626,luis rodriguez martin,ESP,BRUJO,8b,Onsight,SADERNES,ESP
4,1476,Adam Ondra,CZE,MATA HARI,8a+,Onsight,FRANKENJURA,DEU
5,4424,Marcin Wszolek,POL,NUEVE ZETA,8a+,Onsight,CHULILLA,ESP
6,16672,Gonzalo Larrocha,ESP,DNA EXTENSION,8a+,Onsight,KALYMNOS,GRC
7,27822,Michaela Kiersch,USA,LA FEMME BLANCHE,8a+,Onsight,CéüSE,FRA
8,34114,jose luis palao,ESP,TEAM BTR,8a+,Onsight,BIELSA,ESP
9,34114,jose luis palao,ESP,REALIDAD VIRTUAL,8a+,Onsight,POLORIA,ESP


### 5.a) Grado medio y maximo de los ascensos en España de los 10 escaladores NO ESPAÑOLES con mas ascensos en España

In [130]:
from bson.son import SON

#obtenemos los ascensos en España de los escaladores no españoles
result = db.ascensos.aggregate([
    { "$match" : {"pais_escalador": { "$ne": "ESP" }, "pais_risco" : "ESP" } },
    { "$group" : {  "_id": "$id_escalador",
                    "numero_ascensos_esp" : { "$sum": 1 },
                    "dificultad_media_float":{ "$avg": "$id_dificultad"},
                    "dificultad_maxima":{ "$max": "$id_dificultad"},
                    "nombre_escalador" :  {"$addToSet" : "$nombre_escalador"},
                    "pais_escalador" : {"$addToSet" : "$pais_escalador"}
                 }},
    { "$unwind" : "$nombre_escalador" },
    { "$unwind" : "$pais_escalador" },
    { "$sort" : {"numero_ascensos_esp" : -1} },
    { "$limit" : 10 }  
])

df = pd.DataFrame(list(result));
df['dificultad_media'] = df['dificultad_media_float'].round().astype(int)

#mapea dificultadas a grados
mapeaDificultadAGradoFrances(df, ['dificultad_media', 'dificultad_maxima'], ['grado_medio', 'grado_maximo'])

#muestro las columnas del data frame que interesan:
columnas = ['nombre_escalador', 'pais_escalador', 'numero_ascensos_esp','grado_medio', 'grado_maximo']
df[columnas]

,nombre_escalador,pais_escalador,numero_ascensos_esp,grado_medio,grado_maximo
0,Nuno Henriques,PRT,14,6a+,7a
1,"Grzegorz ""Buła"" Golowczyk",POL,9,7b+,8b+
2,Wojtek Pełka,POL,7,7c+/8a,8b
3,philipp kieffer,DEU,7,6c+/7a,7c
4,Gabriel Korbiel,POL,6,7a+/7b,7c
5,Benjamin Thomas,FRA,6,7a,7c+
6,Tieme van Veen,NLD,6,6c+/7a,8a
7,Amber Thornton,GBR,6,5c,6c
8,Kuba Pe,POL,6,7a,7b+
9,"Gonçalo ""Gongas"" Coutinho",PRT,6,5,6a


### 5.b) Grado medio y maximo de los ascensos en España de los 10 escaladores ESPAÑOLES con mas ascensos en España

De manera similar a la pregunta anterior:

In [122]:
from bson.son import SON

#obtenemos los ascensos en España de los escaladores españoles
result = db.ascensos.aggregate([
    { "$match" : {"pais_escalador": "ESP" , "pais_risco" : "ESP" } },
    { "$group" : {  "_id": "$id_escalador",
                    "numero_ascensos_esp" : { "$sum": 1 },
                    "dificultad_media_float":{ "$avg": "$id_dificultad"},
                    "dificultad_maxima":{ "$max": "$id_dificultad"},
                    "nombre_escalador" :  {"$addToSet" : "$nombre_escalador"},
                    "pais_escalador" : {"$addToSet" : "$pais_escalador"}
                 }},
    { "$unwind" : "$nombre_escalador" },
    { "$unwind" : "$pais_escalador" },
    { "$sort" : {"numero_ascensos_esp" : -1} },
    { "$limit" : 10 }  
])
df = pd.DataFrame(list(result));

# crea columna de dificultad media como redondeo de dificultad_media_float
df['dificultad_media'] = df['dificultad_media_float'].round().astype(int)

#mapea dificultadas a grados
mapeaDificultadAGradoFrances(df, ['dificultad_media', 'dificultad_maxima'], ['grado_medio', 'grado_maximo'])

#muestra los resultados
columnas = ['nombre_escalador', 'pais_escalador', 'numero_ascensos_esp','grado_medio', 'grado_maximo']
df[columnas]


,nombre_escalador,pais_escalador,numero_ascensos_esp,grado_medio,grado_maximo
0,Raúl Crespo,ESP,18,6a+/6b,7b
1,Chaken Gómez conde,ESP,16,7c+,8b
2,Adrian Alameda,ESP,14,6c+,8a+
3,Alex Garriga,ESP,13,7b+/7c,8a+
4,Xavier Gatell Romero,ESP,12,7b+,8a
5,jose luis palao,ESP,12,8a+,8b+
6,Gonzalo Larrocha,ESP,10,8a+/8b,9a
7,Tomata Tomata,ESP,10,6c+,7a+
8,luis rodriguez martin,ESP,10,8a/+,8b+
9,Jose Agustí,ESP,10,7c+,8a


### 6.a) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con menos de 3 años de experiencia

In [258]:
from bson.son import SON

result = db.ascensos.aggregate([
        { "$project" : 
            {
                "_id" : 0,
                "id_escalador" : 1,
                "id_dificultad" : 1,
                "tipo_encadenamiento" : 1,
                "experiencia" : {"$subtract":[2017,"$anio_comienzo"]}
            }
        },
        { "$match" : { "tipo_encadenamiento": { "$ne": "Toprope" } }},
        { "$match" : { "$and" : [ {"experiencia": {"$gte": 0 }} , { "experiencia": {"$lt": 3 } }] }},
        { "$group" : {
                "_id" : None,
                "numero_ascensos" : {"$sum" : 1},
                "dificultad_media_float" : {"$avg":"$id_dificultad"},
                "dificultad_maxima" : {"$max":"$id_dificultad"}
            }    
        }
    ])

df = pd.DataFrame(list(result));

# creo la columna de dificultad media como entero
df['dificultad_media'] = df['dificultad_media_float'].round().astype(int)

mapeaDificultadAGradoFrances(df,['dificultad_maxima', 'dificultad_media'], ['grado_maximo', 'grado_medio'])
df[['numero_ascensos', 'grado_maximo', 'grado_medio']]

,numero_ascensos,grado_maximo,grado_medio
0,590,8a,6a/+


### 6.b) Dificultad media y maxima de los ascensos NO "Top Rope" de los escaladores con entre 10 y 30 años de experiencia

In [259]:
from bson.son import SON

result = db.ascensos.aggregate([
        { "$project" : 
            {
                "_id" : 0,
                "id_escalador" : 1,
                "id_dificultad" : 1,
                "tipo_encadenamiento" : 1,
                "experiencia" : {"$subtract":[2017,"$anio_comienzo"]}
            }
        },
        { "$match" : { "tipo_encadenamiento": { "$ne": "Toprope" } }},
        { "$match" : { "$and" : [ {"experiencia": {"$gte": 10 }} , { "experiencia": {"$lt": 30 } }] }},
        { "$group" : {
                "_id" : None,
                "numero_ascensos" : {"$sum" : 1},
                "dificultad_media_float" : {"$avg":"$id_dificultad"},
                "dificultad_maxima" : {"$max":"$id_dificultad"}
            }    
        }
    ])

df = pd.DataFrame(list(result));

# creo la columna de dificultad media como entero
df['dificultad_media'] = df['dificultad_media_float'].round().astype(int)

mapeaDificultadAGradoFrances(df,['dificultad_maxima', 'dificultad_media'], ['grado_maximo', 'grado_medio'])
df[['numero_ascensos', 'grado_maximo', 'grado_medio']]

,numero_ascensos,grado_maximo,grado_medio
0,3818,9c,7a


### 7. Los 10 riscos españoles (o zonas) con mas ascensos por orden decreciente de numero de ascensos

### 8.a) Los 10 sectores españoles con mayor nivel de difcultad media de ascensos ordenadas por orden decreciente de dificultad y por numero de ascensos decreciente

### 8.b) Las 10 sectores españoles con menor nivel de dificultad medio de ascensos ordenadas por orden creciente de dificultad y por numero de ascensos decreciente